In [33]:
!gcloud ai-platform jobs submit training census_prediction_$(date +"%Y%m%d_%H%M%S") \
--staging-bucket gs://cap-4985-mlengine-staging \
--packages='gs://cap-4985-modelmanagement/census/package/prediction/census-batch-prediction-package-0.1.tar.gz' \
--module-name trainer.task \
--region us-central1 \
--runtime-version=1.13 \
--python-version=3.5 \
--scale-tier BASIC \
-- \
--bucket_name cap-4985-modelmanagement \
--output_dir census/output

Job [census_prediction_20200212_145625] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe census_prediction_20200212_145625

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs census_prediction_20200212_145625
jobId: census_prediction_20200212_145625
state: QUEUED


In [ ]:
!gcloud ai-platform jobs submit training census_training_$(date +"%Y%m%d_%H%M%S") \
--staging-bucket gs://cap-4985-mlengine-staging \
--packages='gs://cap-4985-modelmanagement-repo/census/package/census-deploy-package-0.1.tar.gz' \
--module-name trainer.task \
--region us-central1 \
--runtime-version=1.13 \
--python-version=3.5 \
--scale-tier BASIC \
-- \
--bucket_name cap-4985-modelmanagement \
--output_dir census/output

In [25]:
import os
import numpy as np
import pandas as pd
import sys
import json
import logging
from sklearn.externals import joblib
from censusprediction.trainer.helper import gcshelper, bqhelper

censuspredictiontable = 'modelmanagement.censuspredictiontable'

# Specify default parameters.
MODEL_BUCKET = 'cap-4985-modelmanagement'
MODEL_PATH = 'census/output/'
LOCAL_PATH = 'model'

gcshelper.download_from_bucket(MODEL_BUCKET, MODEL_PATH, LOCAL_PATH)
model = joblib.load(LOCAL_PATH + '/model.joblib')

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SelectKBest from version 0.20.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.20.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator FeatureUnion from version 0.20.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own 

In [26]:
df = bqhelper.read_from_bqtable('sb-bigdata-4985-da852265', 'SELECT * FROM `sb-bigdata-4985-da852265.modelmanagement.composer_census_data`')
#inputObjs = np.array(df)

In [27]:
df1 = df.drop(['income_bracket'], axis=1)

In [28]:
op = model.predict(df1)

In [29]:
op

array([False, False, False, ...,  True,  True, False])

In [30]:
df1['prediction_result'] = op

In [34]:
from google.cloud import bigquery


# create a schema for the output BQ table
varbqschema=[
    bigquery.SchemaField(name="age", field_type="INTEGER"),
    bigquery.SchemaField(name="workclass", field_type="STRING"),
    bigquery.SchemaField(name="functional_weight", field_type="INTEGER"),
    bigquery.SchemaField(name="education", field_type="STRING"),
    bigquery.SchemaField(name="education_num", field_type="INTEGER"),
    bigquery.SchemaField(name="marital_status", field_type="STRING"),
    bigquery.SchemaField(name="occupation", field_type="STRING"),
    bigquery.SchemaField(name="relationship", field_type="STRING"),
    bigquery.SchemaField(name="race", field_type="STRING"),
    bigquery.SchemaField(name="sex", field_type="STRING"),
    bigquery.SchemaField(name="capital_gain", field_type="INTEGER"),
    bigquery.SchemaField(name="capital_loss", field_type="INTEGER"),
    bigquery.SchemaField(name="hours_per_week", field_type="INTEGER"),
    bigquery.SchemaField(name="native_country", field_type="STRING"),
    bigquery.SchemaField(name="prediction_result", field_type="BOOLEAN")
]
#write the results and the input data to a new BQ table
bqhelper.write_to_bqtable('sb-bigdata-4985-da852265', censuspredictiontable, varbqschema, df1)